In [4]:
%pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.1 MB/s eta 0:00:00
   ━━━

In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/09_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

In [3]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [ ]:
from pycaret.regression import *
set_rgs = setup(data = train_set, target = "Premium Amount", session_id = 42, fold=3, preprocess=False, remove_outliers=False)
top5 = compare_models(sort = 'RMSE', n_select = 5)

,Description,Value
0,Session id,42
1,Target,Premium Amount
2,Target type,Regression
3,Original data shape,"(960000, 55)"
4,Transformed data shape,"(960000, 55)"
5,Transformed train set shape,"(672000, 55)"
6,Transformed test set shape,"(288000, 55)"
7,Numeric features,54


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:20:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.7945,1.1951,1.0932,0.0153,0.1656,0.1451,5.5500
ridge,Ridge Regression,0.7945,1.1951,1.0932,0.0153,0.1656,0.1451,0.7667
br,Bayesian Ridge,0.7945,1.1951,1.0932,0.0153,0.1656,0.1451,2.8267
omp,Orthogonal Matching Pursuit,0.8000,1.2065,1.0984,0.0059,0.1663,0.1462,0.7500
lasso,Lasso Regression,0.8013,1.2109,1.1004,0.0022,0.1666,0.1465,1.1467
en,Elastic Net,0.8013,1.2108,1.1004,0.0023,0.1666,0.1465,1.2433
llar,Lasso Least Angle Regression,0.8013,1.2109,1.1004,0.0022,0.1666,0.1465,0.7433
knn,K Neighbors Regressor,0.8810,1.4104,1.1876,-0.1621,0.1770,0.1586,951.4967
par,Passive Aggressive Regressor,0.9826,1.5124,1.2222,-0.2459,0.1790,0.1651,15.3667
huber,Huber Regressor,0.9867,1.7490,1.3225,-0.4411,0.1941,0.1758,38.8267


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

stacking_reg = StackingRegressor(
    estimators=[
        ('lr', LinearRegression()),
        ('LGBM', LGBMRegressor(n_estimators=111, max_depth=11, random_state=42)),
        ('xgboost', xgb.XGBRegressor(n_estimators=51, max_depth=8, random_state=42)),
        ('catboost', Catboost(n_estimators=56, depth=7, learning_rate=1, random_state=42)),
    ],
    final_estimator=LinearRegression(),
    n_jobs=-1,
    cv=3
)

stacking_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


StackingRegressor(cv=3,
                  estimators=[('lr', LinearRegression()),
                              ('LGBM',
                               LGBMRegressor(max_depth=11, n_estimators=111,
                                             random_state=42)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            fea...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=8,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=51, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7dacc6a24670>)],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [ ]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0449495669716689

In [ ]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [ ]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0523641976700195

In [ ]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/09_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_13.csv', index=False)